In [3]:
from datetime import datetime as dt
import requests
import pandas as pd
import Tools as t


In [5]:
api_key = 'c06e566dfb583f3a48a89143106b6241'
#sport = 'americanfootball_ncaaf'
sport = 'basketball_ncaab'
region = 'us'
market = 'totals'

In [ ]:
odds_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
    'api_key': api_key,
    'sport': sport,
    'region': region,
    'mkt': market,
}).json()

df = pd.DataFrame.from_dict(odds_response['data'])

df['Date'] = df['commence_time'].apply(lambda x: dt.fromtimestamp(x).replace(hour=0, minute=0, tzinfo=None))
df['Home'] = df['home_team'].apply(lambda x: t.format_team_names(x))
df['left_home'] = df.apply(lambda row: 1 if t.format_team_names(row.teams[0]) == row.Home else 0, axis = 1)
df['Away'] = df.apply(lambda row: t.format_team_names(row.teams[1]) if row.left_home == 1 else t.format_team_names(row.teams[0]), axis = 1)
df['Game'] = df['Date'].astype('str') + ' ' + df['Away'] + ' ' + df['Home']
if market == 'spreads':
    bet = 'Current Spread Home'
else:
    bet = 'Over Under'

df[bet] = df.apply(lambda x:  t.most_frequent([i['odds'][market]['points'][1] for i in x.sites]) if ( x.left_home == 0) else t.most_frequent([i['odds'][market]['points'][0] for i in x.sites]), axis=1)
df.drop(['left_home'], axis=1, inplace=True)

In [18]:
from Odds import get_odds
df = get_odds(version='basketball_nba')
df.head(20)

,sport_key,sport_nice,teams,commence_time,home_team,sites,sites_count,date,away_team,game,total
0,basketball_nba,NBA,"[Brooklyn Nets, Golden State Warriors]",1608681600,Brooklyn Nets,"[{'site_key': 'pointsbetus', 'site_nice': 'Poi...",1,2020-12-22,Golden State Warriors,2020-12-22 Golden State Warriors Brooklyn Nets,222.5
1,basketball_nba,NBA,"[Los Angeles Clippers, Los Angeles Lakers]",1608692400,Los Angeles Lakers,[],0,2020-12-22,Los Angeles Clippers,2020-12-22 Los Angeles Clippers Los Angeles La...,NaN
2,basketball_nba,NBA,"[Charlotte Hornets, Cleveland Cavaliers]",1608768600,Cleveland Cavaliers,[],0,2020-12-23,Charlotte Hornets,2020-12-23 Charlotte Hornets Cleveland Cavaliers,NaN
3,basketball_nba,NBA,"[Indiana Pacers, New York Knicks]",1608768600,Indiana Pacers,[],0,2020-12-23,New York Knicks,2020-12-23 New York Knicks Indiana Pacers,NaN
4,basketball_nba,NBA,"[Miami Heat, Orlando Magic]",1608768600,Orlando Magic,[],0,2020-12-23,Miami Heat,2020-12-23 Miami Heat Orlando Magic,NaN
5,basketball_nba,NBA,"[Philadelphia 76ers, Washington Wizards]",1608768600,Philadelphia 76ers,[],0,2020-12-23,Washington Wizards,2020-12-23 Washington Wizards Philadelphia 76ers,NaN
6,basketball_nba,NBA,"[Boston Celtics, Milwaukee Bucks]",1608769800,Boston Celtics,[],0,2020-12-23,Milwaukee Bucks,2020-12-23 Milwaukee Bucks Boston Celtics,NaN
7,basketball_nba,NBA,"[New Orleans Pelicans, Toronto Raptors]",1608769800,Toronto Raptors,[],0,2020-12-23,New Orleans Pelicans,2020-12-23 New Orleans Pelicans Toronto Raptors,NaN
8,basketball_nba,NBA,"[Atlanta Hawks, Chicago Bulls]",1608772200,Chicago Bulls,[],0,2020-12-23,Atlanta Hawks,2020-12-23 Atlanta Hawks Chicago Bulls,NaN
9,basketball_nba,NBA,"[Memphis Grizzlies, San Antonio Spurs]",1608772200,Memphis Grizzlies,[],0,2020-12-23,San Antonio Spurs,2020-12-23 San Antonio Spurs Memphis Grizzlies,NaN


In [2]:
live_game = df[(df['away_team'].str.lower() == 'Buffalo Bulls'.lower()) & (df['home_team'].str.lower() == 'Miami (OH) Redhawks'.lower())]
live_game['total'].values[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [8]:
df['home_team'].head()

0    CSU Northridge Matadors
1        Miami (OH) RedHawks
2           Wofford Terriers
3        Radford Highlanders
4       Virginia Tech Hokies
Name: home_team, dtype: object

In [17]:
odds_response = requests.get('https://api.the-odds-api.com/v3/sports', params={
    'all':True,
    'api_key': api_key,
}).json()
df = pd.DataFrame.from_dict(odds_response['data'])
df

,key,active,group,details,title,has_outrights
0,americanfootball_ncaaf,True,American Football,US College Football,NCAAF,False
1,americanfootball_nfl,True,American Football,US Football,NFL,False
2,aussierules_afl,False,Aussie Rules,Aussie Football,AFL,False
3,baseball_mlb,False,Baseball,Major League Baseball 🇺🇸,MLB,False
4,basketball_euroleague,True,Basketball,Basketball Euroleague,Basketball Euroleague,False
5,basketball_nba,True,Basketball,US Basketball,NBA,False
6,basketball_ncaab,True,Basketball,US College Basketball,NCAAB,False
7,cricket_asia_cup,False,Cricket,Asia Cup,Asia Cup,False
8,cricket_big_bash,True,Cricket,Big Bash League,Big Bash,False
9,cricket_icc_trophy,False,Cricket,ICC Champions Trophy,ICC Champions,False


In [12]:
import pandas as pd
df = pd.read_csv('test.csv', index_col=['time_stamp'])
print(df)
print(df.first_valid_index() in df.index)
#if not tdf['Game'].iloc[0] in self.df.index:

                     time  period adj_time  away  home  total
time_stamp                                                   
1 0:00      1/1/1900 0:00       1  0:20:00    37    29     66
1 0:01      1/1/1900 0:00       1  0:19:59    37    29     66
1 0:24      1/1/1900 0:00       1  0:19:36    37    29     66
1 0:46      1/1/1900 0:00       1  0:19:14    35    29     64
1 0:59      1/1/1900 0:00       1  0:19:01    35    26     61
...                   ...     ...      ...   ...   ...    ...
1 18:57     1/1/1900 0:18       1  0:01:03     2     0      2
1 19:15     1/1/1900 0:19       1  0:00:45     0     0      0
1 19:27     1/1/1900 0:19       1  0:00:33     0     0      0
1 19:46     1/1/1900 0:19       1  0:00:14     0     0      0
1 20:00     1/1/1900 0:20       1  0:00:00     0     0      0

[84 rows x 6 columns]
True


In [13]:
df.iloc[0]


time        1/1/1900 0:00
period                  1
adj_time          0:20:00
away                   37
home                   29
total                  66
Name: 1 0:00, dtype: object